In [ ]:
# standard imports + comsky
import healpy as hp
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import comsky
%matplotlib inline

In [ ]:
# healpix size. NSIDE must be a power of two. Larger=finer.
NSIDE=2**8
NPIX = hp.nside2npix(NSIDE)

# Number of events to simulate
NGAL = 10000
NISO = 10000
NSRC = 1000

# Source position (crab)
L_SRC = 184.55746
B_SRC = -5.78436


### Example of Compton rings projected on the sky

Here we generate 50 Compton rings projected on the sky, from a simulated source at the center of the projection.  For each photon the orientation and radius of the Compton ring is randomly generated.

In [ ]:
mSrc50 = comsky.utils.MakePointSource(50, NSIDE, 0, 0)
hp.mollview(mSrc50, title="Point Source map")
hp.graticule()

### Compton events from the Galactic Background

Here we simulated NGAL events from a 2 degree band along the Galactic Plane.   The source positions (i.e., true gamma-ray positions) are randomized in the 2 degree band, and then randomly oriented Compton rings are generated from each event.

In [ ]:
mGal = comsky.utils.MakeGalacticBackground(NGAL, NSIDE)
hp.mollview(mGal, title="Galactic Background map")
hp.graticule()

### Compton rings from the isotropic background

Here we simulated NSIO events from the entire sky.  The source positions (i.e., true gamma-ray positions) are randomly selected on the sphere, and then randomly oriented Compton rings are generated from each event.

In [ ]:
mIso = comsky.utils.MakeIsotropicBackground(NISO, NSIDE)
hp.mollview(mIso, title="Isotropic Background map")
hp.graticule()

### Compton rings from a point source

Here we simulated NSRC events from the Crab.  For each event the orientation of the Compton ring is randomly generated.

In [ ]:
mSrc = comsky.utils.MakePointSource(NSRC, NSIDE, L_SRC, B_SRC)
hp.mollview(mSrc, title="Point Source map")
hp.graticule()

### Summed Map

Here we sum the three maps together

In [ ]:
hp.mollview(mSrc+mIso+mGal, title="Fake map (background + galactic plane + point source)")
hp.graticule()

### PSF Map

Here we generate events for a source at the pole.  This will allow us to extract the "PSF" by reading off the map in rings.

In [ ]:
mPSF = comsky.utils.MakePointSource(10000, NSIDE, 0, 90.)
hp.mollview(mPSF, title="PSF")
hp.graticule()

### Convert the PSF map to a set of ALMs for later use in convolutions

In [ ]:
almPSF = hp.sphtfunc.map2alm(mPSF)

### Generate simple map of the Galactic plane to demostrate convolutions

This map is just a 5 degree band centered on the Galactic equator.

In [ ]:
l = 0.
b = 90.
radius=90
width=5
rO=np.deg2rad(radius+0.5*width)
rI=np.deg2rad(radius-0.5*width)
center = hp.pixelfunc.ang2vec(l, b, lonlat=True)
gp_ring = list( set(hp.query_disc(NSIDE, center, rO)) - set(hp.query_disc(NSIDE, center, rI)) )
mGP_True = np.zeros(NPIX)
mGP_True[gp_ring] = 1
hp.mollview(mGP_True, fig=1, title="GP_TRUE_MAP")
hp.graticule()

### Use the ALMs from the PSF map to do the convultions

In [ ]:
mGP_Conv = comsky.utils.ConvolveUsingAlm(mGP_True, almPSF)
hp.mollview(mGP_Conv, fig=1, title="GP_TRUE_MAP")
hp.graticule()

### Extract a 1-dimensional plot of the "PSF" from the PSF map

In [ ]:
thetas = np.degrees(hp.pixelfunc.pix2ang(NSIDE, np.arange(NPIX))[0])

In [ ]:
bins = np.linspace(0., 90., 451)
hist = np.histogram(thetas, bins=bins, weights=mPSF)[0]
sa_hist = np.histogram(thetas, bins=bins)
hist /= sa_hist[0]

In [ ]:
fig = plt.figure()
axs = fig.subplots()
axs.set_xlim(0., 90)
axs.set_xlabel(r"Ang. Sep. [$^\circ$]")
axs.set_ylabel(r"Density / [$0.2^\circ$]")
bin_cent = 0.5*(bins[1:] + bins[0:-1])
axs.plot(bin_cent, hist)

### Make a point source map with only 5 events, to visual the Compton rings

In [ ]:
mSrc = comsky.utils.MakePointSource(50, NSIDE, L_SRC, B_SRC)
hp.mollview(mSrc, title="Point Source map")
hp.graticule()